# Run stargeo queries for hetio-ind diseases

In [1]:
import logging
import os
import gzip

import pandas
import easydict

import starapi.analysis
import starapi.conf

In [2]:
starapi.analysis.logger.propagate = False
starapi.conf.configure('data')

In [3]:
query_df = pandas.read_table('data/queries.tsv')
query_df.head()

,slim_id,slim_name,case_query,control_query
0,DOID:0050156,idiopathic pulmonary fibrosis,IPF == 'IPF',IPF_control == 'IPF_control'
1,DOID:0050741,alcohol dependence,alcoholism == 'alcoholism',alcoholism_control == 'alcoholism_control'
2,DOID:0050742,nicotine dependence,Smoker == 'Smoker',Nonsmoker == 'Nonsmoker'
3,DOID:1024,leprosy,borderline_leprosy == 'borderline_leprosy' or ...,leprosy_control == 'leprosy_control'
4,DOID:10283,prostate cancer,PC == 'PC' or PC_tissue_case == 'PC_tissue_case',PC_control == 'PC_control' or PC_tissue_contro...


In [4]:
errors = list()

for i, row in query_df.iterrows():
    print i, row.slim_id, row.slim_name

    name = row.slim_id.replace(':', '_')
    params = easydict.EasyDict(
        analysis_name = name,
        case_query = row.case_query,
        control_query = row.control_query,
        modifier_query = '',
        min_samples = 3,
    )
    
    directory = os.path.join('data', 'doslim', params.analysis_name)
    if not os.path.isdir(directory):
        os.mkdir(directory)

    logfile = logging.FileHandler(os.path.join(directory, 'log.txt'))
    logfile.setLevel(logging.DEBUG)
    starapi.analysis.logger.addHandler(logfile)
    try:
        sample_df, fold_df, balanced_perm_df, perm_df = starapi.analysis.perform_analysis(params)
        for df in fold_df, balanced_perm_df:
            if df is not None:
                df.reset_index(inplace=True)
        
    except Exception as e:
        sample_df, fold_df, balanced_perm_df, perm_df = None, None, None, None
        errors.append({'slim_id': row.slim_id, 'name': name, 'error': e})
        print(e)
    starapi.analysis.logger.removeHandler(logfile)
    
    # write files
    files = [
        (sample_df, 'samples.tsv', open),
        (fold_df, 'fold_change.tsv.gz', gzip.open),
        (balanced_perm_df, 'balanced_permutation.tsv.gz', gzip.open),
    ]
    for df, filename, opener in files:
        if df is None:
            continue
        path = os.path.join(directory, filename)
        with opener(path, 'wt') as write_file:
            df.to_csv(write_file, index=False, sep='\t', float_format='%.5g')


Started DOID_0050156 analysis
/home/dhimmels/anaconda3/envs/stargeo/lib/python2.7/site-packages/pandas/core/indexing.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
921.44 ms in get_analysis_df("IPF == 'IPF'", "IPF_control == 'IPF_c..., '')
926.25 ms in Loading dataframe for DOID_0050156
Matching sources: 12
Stats: 12 sources, 12 series, 8 platforms, 679 samples
Loading data and calculating fold change for DOID_0050156
Loading data for GSE10667, id = 179
  1.38 s in get_probes(10)
 24.33 s in load_gse( series_id platform_id..., 179, False)
/home/dhimmels/Documents/serg/rephetio/construct/star_api/starapi/analysis.py:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


0 DOID:0050156 idiopathic pulmonary fibrosis
1

Started DOID_0050741 analysis
  1.00 s in get_analysis_df("alcoholism == 'alcoho..., "alcoholism_control ==..., '')
  1.01 s in Loading dataframe for DOID_0050741
Matching sources: 8
Stats: 8 sources, 8 series, 8 platforms, 435 samples
Loading data and calculating fold change for DOID_0050741
Loading data for GSE10356, id = 95
 38.58 ms in get_probes(36)
 22.71 s in load_gse( series_id platform_id..., 95, False)
257.47 ms in get_gene_fold_change(<Gse GSE10356>, False, 0, None)
Loading data for GSE20568, id = 2733
 39.23 ms in get_probes(689)
  2.38 s in load_gse( series_id platform_id..., 2733, False)
317.54 ms in get_gene_fold_change(<Gse GSE20568>, False, 0, None)
Loading data for GSE29555, id = 5002


 DOID:0050741 alcohol dependence
Loading URL ftp://ftp.ncbi.nih.gov/pub/geo/DATA/SeriesMatrix/GSE29555/GSE29555_series_matrix.txt.gz ...
Cache to

  1.27 s in get_probes(389)
 28.23 s in load_gse( series_id platform_id..., 5002, False)
  5.89 s in get_gene_fold_change(<Gse GSE29555>, False, 0, None)
Loading data for GSE44456, id = 8480


...

 DOID:9744 type 1 diabetes mellitus
64

Started DOID_986 analysis
777.64 ms in get_analysis_df("alopecia_areata == 'a..., "alopecia_areata_contr..., '')
781.95 ms in Loading dataframe for DOID_986
Matching sources: 2
Stats: 2 sources, 2 series, 1 platforms, 17 samples
Loading data and calculating fold change for DOID_986
Loading data for GSE45512, id = 8684
  1.54 s in get_probes(4)
 11.32 s in load_gse( series_id platform_id..., 8684, False)
  2.41 s in get_gene_fold_change(<Gse GSE45512>, False, 0, None)
Loading data for GSE58573, id = 10697
  1.62 s in get_probes(4)
  9.51 s in load_gse( series_id platform_id..., 10697, False)
  2.25 s in get_gene_fold_change(<Gse GSE58573>, False, 0, None)
 26.00 s in Load/fold for DOID_986
Meta-Analyzing DOID_986
366.19 s in meta analysis of real data for DOID_986
239.84 ms in meta analysis of permutations for DOID_986
119.92 mks in get_balanced_permutations({'analysis_name': 'DOI...,  C H H_lower H_upper I..., Empty DataFrame Column...)
367.13 s in Meta analysis for DOID_986
DONE DOID_

 DOID:986 alopecia areata
65

Started DOID_9970 analysis
  1.25 s in get_analysis_df("obesity == 'obesity'", "obesity_control == 'o..., '')
  1.25 s in Loading dataframe for DOID_9970
Matching sources: 43
Excluded 23 sources as single-class
Stats: 20 sources, 14 series, 16 platforms, 561 samples
Loading data and calculating fold change for DOID_9970
Loading data for GSE12050, id = 521
  1.50 s in get_probes(332)
 13.50 s in load_gse( series_id platform_id..., 521, False)
  2.58 s in get_gene_fold_change(<Gse GSE12050>, False, 0, None)
Loading data for GSE15524, id = 1461
913.87 ms in get_probes(500)
  8.55 s in load_gse( series_id platform_id..., 1461, False)
  1.60 s in get_gene_fold_change(<Gse GSE15524>, False, 0, None)
Loading data for GSE2508, id = 3892
737.22 ms in get_probes(7)
700.16 ms in get_probes(8)
619.39 ms in get_probes(13)
495.10 ms in get_probes(14)
419.47 ms in get_probes(15)
526.28 ms in get_probes(16)
 28.36 s in load_gse( series_id platform_id..., 3892, False)
  5.13 s in get_gene_fold_change(<

 DOID:9970 obesity


In [5]:
error_df = pandas.DataFrame(errors)
error_df = query_df.merge(error_df)
error_df

,slim_id,slim_name,case_query,control_query,error,name
0,DOID:0050742,nicotine dependence,Smoker == 'Smoker',Nonsmoker == 'Nonsmoker','Int64Index' object has no attribute 'labels',DOID_0050742
1,DOID:11054,urinary bladder cancer,bladder_cancer == 'bladder_cancer',bladder_cancer_control == 'bladder_cancer_cont...,'Int64Index' object has no attribute 'labels',DOID_11054
2,DOID:11714,gestational diabetes,GDM == 'GDM',GDM_control == 'GDM_control','mygene_sym',DOID_11714
3,DOID:12849,autistic disorder,autism == 'autism',autism_control == 'autism_control',zero-size array to reduction operation maximum...,DOID_12849
4,DOID:14221,metabolic syndrome X,MetS == 'MetS',MetS_Control == 'MetS_Control','Int64Index' object has no attribute 'labels',DOID_14221
5,DOID:2998,testicular cancer,testicular_cancer == 'testicular_cancer',testicular_cancer_control == 'testicular_cance...,'Int64Index' object has no attribute 'labels',DOID_2998
6,DOID:3083,chronic obstructive pulmonary disease,COPD == 'COPD',COPD_control == 'COPD_control',"Can't find matrix file for series 6962, platfo...",DOID_3083
7,DOID:4606,bile duct cancer,bile_duct_cancer == 'bile_duct_cancer',bile_duct_cancer_control == 'bile_duct_cancer_...,'Int64Index' object has no attribute 'labels',DOID_4606
8,DOID:635,acquired immunodeficiency syndrome,HIV_Stage1 == 'HIV_Stage1' or HIV_stage2 == 'H...,HIV_healthycontrol == 'HIV_healthycontrol','Int64Index' object has no attribute 'labels',DOID_635


In [6]:
error_df.to_csv('data/errors.csv', index=False)